In [2]:
from scraper import *

In [3]:
driver = initialize_otodom_scraper()

In [5]:
url = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?limit=36&extras=%5BAIR_CONDITIONING%2CGARDEN%5D&by=DEFAULT&direction=DESC&viewType=listing&page=1'
a = get_announcements_links(driver,url)

In [6]:
len(a)

99

In [ ]:
//p[@data-cy="listing-item-title"]

In [4]:
def get_links_titles(
    first_page_url : str,
    sleep_length : float = 1,
    return_df : bool = True,
    save_as_csv : bool = True,
    csv_file_name_prefix : str = 'available_ads',
    csv_destination_path : str = 'data_raw') -> pd.DataFrame:
    
    
    driver = initialize_otodom_scraper(sleep_length)
    
    # Container
    announcements = []

    # Loop through otodom using link with prefilled search
    i=1
    while True:
        site = re.sub('&page=[^&]*', f'&page={i}', first_page_url)
        driver.get(site)

        # Iter until page without new result appears
        try:
            driver.find_element(By.XPATH, '//div[@data-cy="no-search-results"]')
            break
        except NoSuchElementException:
            i+=1
        
        # Go to the down of the page to load all announcements
        sleep(sleep_length)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(sleep_length)
        
        # Get announcements links and titles
        links = driver.find_elements(By.XPATH, '//a[@data-cy="listing-item-link"]')
        titles = driver.find_elements(By.XPATH, '//p[@data-cy="listing-item-title"]')
        
        # Collect data
        for link, title in zip(links, titles):
            announcements.append({
                "link": link.get_attribute('href'),
                "title": title.text.strip()
            })

    # Remove duplicates; return result
    df = pd.DataFrame(announcements).drop_duplicates().reset_index(drop=True)
    current_date = datetime.now().strftime("%Y_%m_%d")
    df['date'] = current_date
    
    # Save as csv
    if save_as_csv:
        file_name = f"{csv_file_name_prefix}_{current_date}.csv"
        path = f'{csv_destination_path}/{file_name}'
        df.to_csv(path,
                  encoding='utf-8',
                  index=False)
        
    return df if return_df else None

In [8]:
url = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?limit=36&extras=%5BAIR_CONDITIONING%2CGARDEN%5D&by=DEFAULT&direction=DESC&viewType=listing&page=1'
df = get_links_titles(url)

In [28]:
df

,link,title,date
0,https://www.otodom.pl/pl/oferta/bezposrednio-w...,Bezpośrednio Wynajmę nowe mieszkanie SUPER,2024_12_21
1,https://www.otodom.pl/pl/oferta/prywat-4-pok-7...,"Prywat.4 pok.77m2.KOPERNIK,Ośr SOLEC,Bulwary,M...",2024_12_21
2,https://www.otodom.pl/pl/oferta/od-2025-ciche-...,od 2025 Ciche | ogród | SMART | LUX łóżko | klima,2024_12_21
3,https://www.otodom.pl/pl/oferta/nowoczesny-apa...,nowoczesny apartament z sauna i widokiem na wodę,2024_12_21
4,https://www.otodom.pl/pl/oferta/apartament-sau...,apartament sauna duży taras widok na wodę,2024_12_21
...,...,...,...
93,https://www.otodom.pl/pl/oferta/apartament-na-...,Apartament Na Mordorze|Ogród|Sauna|Siłownia|Taras,2024_12_21
94,https://www.otodom.pl/pl/oferta/apartament-z-t...,Apartament z tarasem i ogrodem,2024_12_21
95,https://www.otodom.pl/pl/oferta/apartament-na-...,Apartament na Mordorze|Ogród|Sauna|Siłownia|Taras,2024_12_21
96,https://www.otodom.pl/pl/oferta/elegancki-segm...,,2024_12_21


In [19]:
df_2 = run_otodom_scraper(first_page_url = url, sleep_length=2)

In [26]:
df_2

,title,rent_price,area_room_num,floor,ogrzewanie,flat_condition,available_from,deposit,advertiser_type,additional_information,...,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,announcement_date,adv_description,link
0,Extra Apartament 120 m2,8 500 zł\n/miesiąc,120m²\n5 pokoi\ntylko dla niepalących,parter/2,gazowe,do zamieszkania,2024-12-20,8 500 zł,prywatny,balkon\nogródek\ngaraż/miejsce parkingowe\ntyl...,...,2020,nie,dom wolnostojący,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,telewizja kablowa\ninternet,drzwi / okna antywłamaniowe\ndomofon / wideofo...,(Aktualizacja: 20.12.2024\nDodano: 20.12.2024\...,Do wynajęcia apartament o luksusowym standardz...,https://www.otodom.pl/pl/oferta/extra-apartame...
1,Nowoczesne 2 pokojowe mieszkanie na Wilanowie,4 100 zł\n/miesiąc\n+ Czynsz 700 zł,45m²\n2 pokoje\ntylko dla niepalących,1,miejskie,do zamieszkania,2025-01-01,8 200 zł,prywatny,balkon\nogródek\ngaraż/miejsce parkingowe\ntyl...,...,2018,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,None,drzwi / okna antywłamaniowe\ndomofon / wideofon,(Aktualizacja: 21.12.2024\nDodano: 16.05.2021\...,Nowoczesne 2-pokojowe mieszkanie z garażem na ...,https://www.otodom.pl/pl/oferta/nowoczesne-2-p...
2,Zielona Białołęka | 4-pok | Taras +Ogródek | K...,4 500 zł\n/miesiąc\n+ Czynsz 1 700 zł,105m²\nWynajmę również studentom\n4 pokoje\nty...,parter/4,miejskie,do zamieszkania,None,6 200 zł,prywatny,balkon\nogródek\ntaras\ngaraż/miejsce parkingo...,...,2009,tak,blok,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,telewizja kablowa\ninternet,system alarmowy\ndrzwi / okna antywłamaniowe\n...,(Aktualizacja: 15.12.2024\nDodano: 3.11.2024\n...,NAJEM OKAZJONALNY*\nZielona Białołęka\nOsiedle...,https://www.otodom.pl/pl/oferta/zielona-bialol...
3,nowoczesny apartament z sauna i widokiem na wodę,9 000 zł\n/miesiąc\n+ Czynsz 2 000 zł,130m²\nWynajmę również studentom\n4 pokoje,3/3,miejskie,do zamieszkania,None,18 000 zł,prywatny,balkon\nogródek\ntaras\ngaraż/miejsce parkingo...,...,2015,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,telewizja kablowa\ninternet,drzwi / okna antywłamaniowe\ndomofon / wideofon,(Aktualizacja: 21.12.2024\nDodano: 1.12.2024\n...,Do wynajęcia piękny apartament w inwestycji At...,https://www.otodom.pl/pl/oferta/nowoczesny-apa...
4,apartament sauna duży taras widok na wodę,9 000 zł\n/miesiąc\n+ Czynsz 2 000 zł,130m²\n4 pokoje,3/3,None,do zamieszkania,None,None,prywatny,balkon\nogródek\ntaras\ngaraż/miejsce parkingowe,...,2015,tak,blok,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,telewizja kablowa\ninternet\ntelefon,drzwi / okna antywłamaniowe\ndomofon / wideofon,(Aktualizacja: 21.12.2024\nDodano: 1.11.2024\n...,Do wynajęcia piękny apartament w inwestycji At...,https://www.otodom.pl/pl/oferta/apartament-sau...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Apartament Na Mordorze|Ogród|Sauna|Siłownia|Taras,16 000 zł\n/miesiąc,105m²\n4 pokoje,parter/8,miejskie,do zamieszkania,2024-01-04,None,biuro nieruchomości,ogródek\ntaras\ngaraż/miejsce parkingowe,...,2023,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,meble\npiekarnik\nlodówka\npralka\ntelewizor\n...,internet\ntelewizja kablowa,drzwi / okna antywłamaniowe\ndomofon / wideofon,(Aktualizacja: 20.12.2024\nDodano: 28.03.2024\...,We also speak: UA|ENG|RU\n\nGustownie wykończo...,https://www.otodom.pl/pl/oferta/apartament-na-...
94,Apartament z tarasem i ogrodem,10 000 zł\n/miesiąc\n+ Czynsz 800 zł,85m²\n3 pokoje,parter/2,miejskie,do zamieszkania,None,10 000 zł,biuro nieruchomości,balkon\nogródek\ntaras\ngaraż/miejsce parkingo...,...,None,tak,apartamentowiec,teren zamknięty\nmonitoring / ochrona,zmywarka\nlodówka\nmeble\npiekarnik\nkuchenka\...,telewizja kablo

In [30]:
result = pd.merge(df_2, df, how="left", on=['link', 'title'])

In [7]:
driver.get(a[5])

In [12]:
get_location_from_map(driver)

{'latitude': '52.23338', 'longitude': '20.95718', 'approximate': False}

In [11]:
def get_text_from_class(driver, class_name):
    try:
        text = driver.find_element(By.XPATH, f"//div[@class='{class_name}']").text
    except NoSuchElementException:
        text = None
    return text

In [13]:
class_name = 'css-wqvm7k ef3kcx01'
driver.find_element(By.XPATH, f"//h1[@class='{class_name}']").text

'Mieszkanie 130 m2 , 5 pok 2 miejsca postojowe'

In [230]:
get_text_from_class(driver, 'css-1ifvn3m ef3kcx02')

'16 000 zł\n/miesiąc'

In [232]:
get_text_from_class(driver, 'css-58w8b7 eezlw8k0')

'105m²\n4 pokoje'

In [234]:
get_text_from_class(driver, 'css-gg4vpm e2md81j0')

'Aktualizacja: 20.11.2024\nDodano: 28.03.2024\nID: 65272925\nZgłoś'

In [236]:
get_text_from_class(driver, 'css-pla15i e5h9f1b2')

'ul. Domaniewska, Służewiec, Mokotów, Warszawa, mazowieckie'

In [17]:
def get_text_from_main_table(driver, label_name):
    try:
        xpath = (
            "//div[h4[contains(text(), 'Mieszkanie na wynajem')]]"
            f"/div/div[p[contains(text(), '{label_name}')]]/p[2]"
        )
        value_element = driver.find_element(By.XPATH, xpath)
        text = value_element.text
    except NoSuchElementException:
        text = None
    return text if text != 'brak informacji' else None

In [214]:
get_text_from_main_table(driver, 'Ogrzewanie')

'miejskie'

In [216]:
get_text_from_main_table(driver, 'Piętro')

'parter/8'

In [240]:
get_text_from_main_table(driver, 'Stan wykończenia')

'do zamieszkania'

In [218]:
get_text_from_main_table(driver, 'Dostępne od')

'2024-01-04'

In [238]:
get_text_from_main_table(driver, 'Kaucja')

In [226]:
get_text_from_main_table(driver, 'Typ ogłoszeniodawcy')

'biuro nieruchomości'

In [228]:
get_text_from_main_table(driver, 'Informacje dodatkowe')

'ogródek\ntaras\ngaraż/miejsce parkingowe'

In [19]:
def get_text_from_supp_table(driver, table_name, label_name):
    try:
        xpath = (
            f"//div[header/p[contains(text(), '{table_name}')]]"
            f"/div/div/div[p[contains(text(), '{label_name}')]]/p[2]"
        )
        value_element = driver.find_element(By.XPATH, xpath)
        text = value_element.text
    except NoSuchElementException:
        text = None
    return text if text != 'brak informacji' else None

In [172]:
get_text_from_supp_table(driver, 'Wyposażenie', 'Wyposażenie')

'meble\npiekarnik\nlodówka\npralka\ntelewizor\nzmywarka\nklimatyzacja'

In [174]:
get_text_from_supp_table(driver, 'Wyposażenie', 'Zabezpieczenia')

'drzwi / okna antywłamaniowe\ndomofon / wideofon'

In [176]:
get_text_from_supp_table(driver, 'Wyposażenie', 'Media')

'internet\ntelewizja kablowa'

In [178]:
get_text_from_supp_table(driver, 'Budynek i materiały', 'Rok budowy')

'2023'

In [180]:
get_text_from_supp_table(driver, 'Budynek i materiały', 'Winda')

'tak'

In [182]:
get_text_from_supp_table(driver, 'Budynek i materiały', 'Rodzaj zabudowy')

'apartamentowiec'

In [184]:
get_text_from_supp_table(driver, 'Budynek i materiały', 'Bezpieczeństwo')

'teren zamknięty\nmonitoring / ochrona'

In [260]:
driver.find_element(By.XPATH, "//header[@role='button']//p[text()='Budynek i materiały']").click()

In [262]:
driver.find_element(By.XPATH, "//header[@role='button']//p[text()='Wyposażenie']").click()

In [264]:
driver.find_element(By.XPATH, f"//button[.//span[contains(text(), 'Pokaż więcej')]]").click()

In [21]:
def get_adv_description(
    driver : selenium.webdriver.chrome.webdriver.WebDriver) -> str:
    """
    Retrieves the full advertisement description from a web page by clicking a button to expand the text if necessary.

    Args:
    - driver (selenium.webdriver.chrome.webdriver.WebDriver): A Selenium WebDriver instance for Chrome, used to control the browser.

    Returns:
    - str: The full text of the advertisement description,
    or `None` if the element is not found.
    """

    # Find the button to load full description
    button = None
    try:
        button = driver.find_element(By.XPATH, f"//button[.//span[contains(text(), 'Pokaż więcej')]]")

    except NoSuchElementException:
        pass

    if button is not None:
        desc_location = driver.find_element(By.XPATH, f"//h4[contains(text(), 'Opis')]")
        desc_location.location_once_scrolled_into_view
        button.click()

    # Get description
    try:
        adv_description = driver.find_element(By.XPATH, f'//div[@data-cy="adPageAdDescription"]').text
    except NoSuchElementException:
        return None
    
    return adv_description

In [38]:
def scrape_single_announcement(
    driver : selenium.webdriver.chrome.webdriver.WebDriver,
    announcements_links : str,
    sleep_length : float = 1) -> dict:
    """
    Scrapes data from a single apartment for rent announcement on a webpage using a given web driver.

    Parameters:
    driver (selenium.webdriver.chrome.webdriver.WebDriver): The Selenium WebDriver instance to interact with the webpage.

    Returns:
    dict: A dictionary containing scraped data of various attributes of the rent announcement.
    """
    
    driver.get(herf)
    sleep(sleep_length)
    
    try:
        driver.find_element(By.XPATH, "//div[@data-cy='expired-ad-alert']")
        return None
    except NoSuchElementException:
        pass
    
    scrape_dict = {
        'rent_price' : get_text_from_class(driver, 'css-1ifvn3m ef3kcx02'),
        'area_room_num' : get_text_from_class(driver, 'css-58w8b7 eezlw8k0'),
        'floor': get_text_from_main_table(driver, 'Piętro'),
        'ogrzewanie':get_text_from_main_table(driver, 'Ogrzewanie'),
        'flat_condition': get_text_from_main_table(driver, 'Stan wykończenia'),
        'available_from': get_text_from_main_table(driver, 'Dostępne od'),
        'deposit': get_text_from_main_table(driver, 'Kaucja'),
        'advertiser_type': get_text_from_main_table(driver, 'Typ ogłoszeniodawcy'),
        'additional_information': get_text_from_main_table(driver, 'Informacje dodatkowe'),
        'location': get_text_from_class(driver, 'css-70qvj9 e42rcgs0')
    }
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(sleep_length/2)
    try:
        map_element = driver.find_element(By.ID, 'map')
    except NoSuchElementException:
        pass
    map_element.location_once_scrolled_into_view
    sleep(sleep_length)
    geo_location = get_location_from_map(driver)
    scrape_dict['latitude'] = geo_location['latitude']
    scrape_dict['longitude'] = geo_location['longitude']
    scrape_dict['approximate_coordinates'] = geo_location['approximate']
    
    try:
        driver.find_element(By.XPATH, "//header[@role='button']//p[text()='Budynek i materiały']").click()
    except NoSuchElementException:
        pass
    sleep(sleep_length/2)
    scrape_dict['year_of_construction'] = get_text_from_supp_table(driver, 'Budynek i materiały', 'Rok budowy')
    scrape_dict['elevator'] = get_text_from_supp_table(driver, 'Budynek i materiały', 'Winda')
    scrape_dict['building_type'] = get_text_from_supp_table(driver, 'Budynek i materiały', 'Rodzaj zabudowy')
    scrape_dict['security'] = get_text_from_supp_table(driver, 'Budynek i materiały', 'Bezpieczeństwo')
    
    try:
        driver.find_element(By.XPATH, "//header[@role='button']//p[text()='Wyposażenie']").click()   
    except NoSuchElementException:
        pass
    sleep(sleep_length/2)
    scrape_dict['equipment'] = get_text_from_supp_table(driver, 'Wyposażenie', 'Wyposażenie')
    scrape_dict['utilities'] = get_text_from_supp_table(driver, 'Wyposażenie', 'Media')
    scrape_dict['safeguards'] = get_text_from_supp_table(driver, 'Wyposażenie', 'Zabezpieczenia')
    
    scrape_dict['announcement_date'] = get_text_from_class(driver, 'css-gg4vpm e2md81j0'),
    scrape_dict['adv_description'] = get_adv_description(driver)
    
    return scrape_dict

In [29]:
a[36]

'https://www.otodom.pl/pl/oferta/2-pokoje-w-royal-residence-wilanow-ID,4tni0'

In [40]:
herf = a[40]
scrape_single_announcement(driver, herf, 1)

{'rent_price': '4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł',
 'area_room_num': '47m²\n2 pokoje',
 'floor': 'parter/9',
 'ogrzewanie': 'miejskie',
 'flat_condition': 'do zamieszkania',
 'available_from': '2023-06-01',
 'deposit': '6 500 zł',
 'advertiser_type': 'biuro nieruchomości',
 'additional_information': 'ogródek\ngaraż/miejsce parkingowe',
 'location': 'ul. Józefa Mehoffera, Henryków, Białołęka, Warszawa, mazowieckie',
 'latitude': '52.33767',
 'longitude': '20.96563',
 'approximate_coordinates': False,
 'year_of_construction': '2018',
 'elevator': 'nie',
 'building_type': 'blok',
 'security': 'teren zamknięty',
 'equipment': 'meble\npiekarnik\nlodówka\npralka\ntelewizor\nzmywarka\nklimatyzacja',
 'utilities': 'telefon\ninternet\ntelewizja kablowa',
 'safeguards': 'domofon / wideofon',
 'announcement_date': ('Aktualizacja: 12.12.2024\nDodano: 12.11.2024\nID: 66114116\nZgłoś',),
 'adv_description': 'Mamy przyjemność zaprezentować Państwu nowoczesne 2-pokojowe mieszkanie o powierzchni 5

In [44]:
def scrape_otodom_announcements(
    driver : selenium.webdriver.chrome.webdriver.WebDriver,
    announcements_links : list,
    sleep_length : float = 1,
    save_as_csv : bool = True,
    csv_file_name_prefix : str = 'otodom',
    csv_destination_path : str = 'data_raw') -> pd.DataFrame:
    """
    Scrapes apartment for rent announcements from provided links on the Otodom website, compiles the data into a DataFrame, 
    and optionally saves it as a CSV file.

    Parameters:
    - driver (selenium.webdriver.chrome.webdriver.WebDriver): Selenium WebDriver for web page interaction.
    - announcements_links (list): List of URLs to individual announcements to be scraped.
    - sleep_length (float, optional): Time in seconds to pause between actions to simulate human behavior and ensure
      page content is loaded. Defaults to 0.5.
    - save_as_csv (bool, optional): If True, saves the compiled DataFrame as a CSV file. Defaults to True.
    - csv_file_name_prefix (str, optional): Prefix for the CSV file name. Defaults to 'otodom'.
    - csv_destination_path (str, optional): Directory path where the CSV file will be saved. Defaults to 'data_raw'.

    Returns:
    - pd.DataFrame: DataFrame containing scraped data from each announcement link.
    """
    # Container
    scraped_dicts = []

    # Loop through scraped announcements links
    for i in announcements_links:
        print(i)
        single_announcement = scrape_single_announcement(driver, i, sleep_length)
        single_announcement['link'] = i
        scraped_dicts.append(single_announcement)

    # Concatenate dictionaries
    concat_dict = defaultdict(list)
    for dictionary in scraped_dicts:
        for key, value in dictionary.items():
            concat_dict[key].append(value)

    # Converting defaultdict to pd.DataFrame
    df = pd.DataFrame(dict(concat_dict))

    # Save df as csv file with current date suffix
    if save_as_csv:
        current_date = datetime.now().strftime("%Y_%m_%d")
        file_name = f"{csv_file_name_prefix}_{current_date}.csv"
        path = f'{csv_destination_path}/{file_name}'
        df.to_csv(path,
                  encoding='utf-8',
                  index=False)
    return df

In [46]:
scrape_otodom_announcements(driver, a[:20])

https://www.otodom.pl/pl/oferta/zlota-44-160-m2-3-sypialnie-ID42aaV
https://www.otodom.pl/pl/oferta/od-2025-ciche-ogrod-smart-lux-lozko-klima-ID4tTJU
https://www.otodom.pl/pl/oferta/premium-dwupokojowe-mieszkanie-z-ogrodkiem-100-m2-ID4pwF2
https://www.otodom.pl/pl/oferta/2-pokoje-w-royal-residence-wilanow-ID4tni0
https://www.otodom.pl/pl/oferta/idealne-mieszkanie-dla-pary-biznesowy-mokotow-ID4tSlJ
https://www.otodom.pl/pl/oferta/3-pokojowe-mieszkanie-na-powislu-ID4t0Il
https://www.otodom.pl/pl/oferta/trzypokojowe-mieszkanie-miasteczko-wilanow-ID4tWMz
https://www.otodom.pl/pl/oferta/rakow-83-m-wysoki-standard-klima-ogrod-ID4tSAG
https://www.otodom.pl/pl/oferta/nowe-2-pokojowe-mieszkanie-niedaleko-metro-mlociny-ID4tKdI
https://www.otodom.pl/pl/oferta/atrakcyjny-nowoczesny-apartament-mokotow-ID4tro4
https://www.otodom.pl/pl/oferta/nowe-mieszkanie-z-ogrodkiem-w-poblizu-hali-koszyki-ID4ozIj
https://www.otodom.pl/pl/oferta/zielona-bialoleka-4-pok-taras-ogrodek-klima-ID4thjo
https://www.otodo

,rent_price,area_room_num,floor,ogrzewanie,flat_condition,available_from,deposit,advertiser_type,additional_information,location,...,year_of_construction,elevator,building_type,security,equipment,utilities,safeguards,announcement_date,adv_description,link
0,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/zlota-44-160-m...
1,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/od-2025-ciche-...
2,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/premium-dwupok...
3,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/2-pokoje-w-roy...
4,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/idealne-mieszk...
5,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/3-pokojowe-mie...
6,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/trzypokojowe-m...
7,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/rakow-83-m-wys...
8,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.2024\...,Mamy przyjemność zaprezentować Państwu nowocze...,https://www.otodom.pl/pl/oferta/nowe-2-pokojow...
9,4 500 zł\n/miesiąc\n+ Czynsz 1 000 zł,47m²\n2 pokoje,parter/9,miejskie,do zamieszkania,2023-06-01,6 500 zł,biuro nieruchomości,ogródek\ngaraż/miejsce parkingowe,"ul. Józefa Mehoffera, Henryków, Białołęka, War...",...,2018,,,,meble,,,(Aktualizacja: 12.12.2024\nDodano: 12.11.202